In [ ]:
import numpy as np
import soundfile as sf
from scipy.signal import correlate

# -----------------------------
# STEP 1: Load CLEAN input signal
# -----------------------------
x, fs = sf.read('impulse_response2.wav')
x = x[:,0] if x.ndim > 1 else x
x /= np.max(np.abs(x))  # normalize

# -----------------------------
# STEP 2: Load room recordings
# -----------------------------
recordings = []
for k in range(1, 6):
    y, _ = sf.read(f'Test_echo{k}.wav')
    y = y[:,0] if y.ndim > 1 else y
    recordings.append(y)

# -----------------------------
# STEP 3: Time-align recordings
# -----------------------------
aligned = []

for y in recordings:
    corr = correlate(y, x, mode='full')
    shift = np.argmax(corr) - len(x) + 1

    if shift > 0:
        y = y[shift:]
    else:
        y = y[:shift]

    aligned.append(y)

# Trim to same length
min_len = min(len(y) for y in aligned)
Y = np.array([y[:min_len] for y in aligned])

# -----------------------------
# STEP 4: Average recordings
# -----------------------------
y_avg = np.mean(Y, axis=0)

# -----------------------------
# STEP 5: Window early response (VERY IMPORTANT)
# -----------------------------
window = int(0.3 * fs)   # first 300 ms
y_avg = y_avg[:window]

# -----------------------------
# STEP 6: Deconvolution (FFT)
# -----------------------------
N = len(y_avg) + len(x)
X = np.fft.fft(x, N)
Yf = np.fft.fft(y_avg, N)

epsilon = 1e-6
H = Yf / (X + epsilon)

h = np.real(np.fft.ifft(H))

# Normalize
h /= np.max(np.abs(h))

# -----------------------------
# STEP 7: Save impulse response
# -----------------------------
sf.write('impulse_response3.wav', h, fs)

print("Impulse response saved successfully.")


LibsndfileError: Error opening 'Test_response2.wav': System error.